In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from keras import Sequential, layers, Input, regularizers
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [2]:
df = pd.read_csv('../data/data_clean.csv')
df.head(5)

/var/folders/nj/s1c8f0tj3ksfjdg8xyxr6rdc0000gn/T/ipykernel_65231/2597316256.py:1: DtypeWarning: Columns (38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/data_clean.csv')


,uid,id,nature,acheteur_id,acheteur_nom,acheteur_siren,titulaire_id,titulaire_typeIdentifiant,titulaire_nom,titulaire_siren,...,codeCPV_2,codeCPV_3,codeCPV_4,codeCPV_5,codeCPV_2_3,annee,acheteur_tranche_effectif,acheteur_categorie,titulaire_tranche_effectif,titulaire_categorie
0,2154005160001320242024-LOT04,20242024-LOT04,Marché,21540051600013,COMMUNE DE BATILLY,215400516.0,78885470100018,SIRET,ACOMETAL,788854701.0,...,45000000,45200000,45260000,45261000,45200000,2024,12,PME,03,PME
1,243500667002882021M226MO,2021M226MO,Marché,24350066700288,CC VAL D'ILLE-AUBIGNE,243500667.0,38373211200032,SIRET,UNIVERS,383732112.0,...,71000000,71200000,71200000,71200000,71200000,2021,22,PME,03,PME
2,249710047000472024SS_PRD_TRV,2024SS_PRD_TRV,Marché,24971004700047,COMMUNAUTE DE COMMUNES DE MARIE GALANTE,249710047.0,43387249600016,SIRET,COTRAM B.T.P.,433872496.0,...,45000000,45200000,45230000,45232000,45200000,2024,12,PME,12,PME
3,6254801990001124-0806-L2,24-0806-L2,Marché,62548019900011,"LA MAISON POUR TOUS, SOCIETE ANONYME COOPERATI...",625480199.0,55204695502544,SIRET,ENGIE ENERGIE SERVICES,552046955.0,...,50000000,50700000,50720000,50720000,50000000,2024,22,ETI,53,GE
4,20002563300013202424011BCR,202424011BCR,Marché,20002563300013,SI DE RESTAURATION COLLECTIVE,200025633.0,47698032100238,SIRET,POMONA EPISAVEURS,476980321.0,...,15000000,15800000,15800000,15800000,15000000,2024,12,PME,42,GE


In [3]:
from scripts.data_cleaner import filter_top_cpv_categories

df = filter_top_cpv_categories(df, top_n=400, cpv_column='codeCPV_4')

Filtered from 1222 to 400 CPV categories, keeping 277081 rows out of 286850


In [4]:
len(df['codeCPV_4'].unique())

400

In [5]:
df.drop(df[df['montant'] > 999999].index, inplace=True)
df.shape

(236269, 42)

In [6]:
y = np.log1p(df['montant'])
y.head()

0    11.292279
1    10.882925
4    13.235694
5    11.350889
6    12.044412
Name: montant, dtype: float64

In [7]:
bins = np.linspace(y.min(), y.max(), 10)
bins = np.delete(bins, 1)

#bins = bins[2:]

# Créer les étiquettes correspondantes
labels = list(range(1, len(bins)))

# Ajouter la colonne fourchette_de_prix
df['bins'] = pd.cut(y, bins=bins, labels=labels, include_lowest=True, right=True)

df.shape

(236269, 43)

In [8]:
df['bins'] = df['bins'].astype('int')
df = df.drop(columns='montant')
df.columns

Index(['uid', 'id', 'nature', 'acheteur_id', 'acheteur_nom', 'acheteur_siren',
       'titulaire_id', 'titulaire_typeIdentifiant', 'titulaire_nom',
       'titulaire_siren', 'objet', 'codeCPV', 'procedure', 'dureeMois',
       'dateNotification', 'datePublicationDonnees', 'formePrix',
       'attributionAvance', 'offresRecues', 'marcheInnovant', 'ccag',
       'sousTraitanceDeclaree', 'typeGroupementOperateurs', 'tauxAvance',
       'origineUE', 'origineFrance', 'lieuExecution_code',
       'lieuExecution_typeCode', 'idAccordCadre', 'source_open_data',
       'codeCPV_FR', 'codeCPV_2', 'codeCPV_3', 'codeCPV_4', 'codeCPV_5',
       'codeCPV_2_3', 'annee', 'acheteur_tranche_effectif',
       'acheteur_categorie', 'titulaire_tranche_effectif',
       'titulaire_categorie', 'bins'],
      dtype='object')

In [9]:
X = df.drop(columns=['bins', 'titulaire_tranche_effectif', 'titulaire_categorie'])
y = df['bins']

In [10]:
X.shape

(236269, 39)

In [11]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 236269 entries, 0 to 286849
Data columns (total 39 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   uid                        236269 non-null  object 
 1   id                         236269 non-null  object 
 2   nature                     236269 non-null  object 
 3   acheteur_id                236269 non-null  int64  
 4   acheteur_nom               236113 non-null  object 
 5   acheteur_siren             236134 non-null  float64
 6   titulaire_id               236269 non-null  object 
 7   titulaire_typeIdentifiant  236269 non-null  object 
 8   titulaire_nom              229976 non-null  object 
 9   titulaire_siren            235422 non-null  float64
 10  objet                      236269 non-null  object 
 11  codeCPV                    236269 non-null  object 
 12  procedure                  236269 non-null  object 
 13  dureeMois                  236269 

In [12]:
df[['acheteur_tranche_effectif', 'acheteur_categorie']] = df[['acheteur_tranche_effectif', 'acheteur_categorie']].fillna('null')
df[['acheteur_tranche_effectif', 'acheteur_categorie']].isna().sum()

acheteur_tranche_effectif    0
acheteur_categorie           0
dtype: int64

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0, stratify=X['codeCPV_4'])

In [14]:
X_train.shape

(189015, 39)

In [15]:
from scripts.preprocess_pipeline import create_pipeline

numerical_columns = ['dureeMois', 'offresRecues', 'annee']

binary_columns = ['sousTraitanceDeclaree', 'origineFrance',
                          'marcheInnovant', 'idAccordCadre']

categorical_columns = ['procedure', 'nature', 'formePrix', 'ccag',
                               'typeGroupementOperateurs', 'tauxAvance_cat',
                               'codeCPV_4', 'acheteur_tranche_effectif', 'acheteur_categorie']

pipeline = create_pipeline(numerical_columns, binary_columns, categorical_columns)

In [16]:
X_train_preproc = pipeline.fit_transform(X_train)
X_train_preproc.shape

(189015, 470)

In [17]:
from keras.utils import to_categorical
y_train_cat = to_categorical(y_train)
y_train_cat.shape

(189015, 9)

In [ ]:
# Dimensions
input_dim = 470
num_classes = 9

# Modèle MLP
model = Sequential()

model.add(Input(shape=(input_dim,)))

model.add(layers.Dense(512, activation='relu', input_shape=(input_dim,)))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.3))

model.add(layers.Dense(256, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.3))

model.add(layers.Dense(128, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.2))

model.add(layers.Dense(num_classes, activation='softmax'))  # Sortie pour classification

# Compilation
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',  # ou 'categorical_crossentropy' si y est one-hot
    metrics=['accuracy']
)

# Affichage du résumé du modèle
model.summary()